## _Check Colab GPU

In [ ]:
# _Check Colab GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

## _Set Memory Growth

In [ ]:
# _Set Memory Growth

from tensorflow import config, test
from tensorflow import __version__ as tf_version

physical_devices = config.list_physical_devices('GPU')
config.experimental.set_memory_growth(physical_devices[0], True)

print(physical_devices)
print(tf_version)
print(f'tf is using cuda: {test.is_built_with_cuda()}')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.8.0
True


## _Load from Drive

In [ ]:
# _Load from Drive
folder_path = '/content/drive/MyDrive/London/term_02/coding_3/week_02'
!cp -av {folder_path} /content/

## _Save to Drive

In [ ]:
# _Save to Drive
folder_name = 'week_02'
based_path = '/content/drive/MyDrive/London/term_02/coding_3'
!cp -av /content/{folder_name}  {based_path}

## _Download Kaggle Dataset

In [ ]:
# _Download Kaggle Dataset
# kaggle.json is the file generated from kaggle.com account setting.
!mkdir .kaggle/
!cp /content/drive/MyDrive/Colab_Notebooks/kaggle.json /content/.kaggle/
!mv .kaggle /root/

In [ ]:
!kaggle datasets download -d koryakinp/fingers

## _Crop Images from Folder and Save to New Folder

In [ ]:
# _Crop images from folder and save to new folder

from PIL import Image
import os, os.path

In [ ]:
# _Crop images from folder and save to new folder

imgs = []
path = "/content/fingers/train"
valid_images = [".jpg",".gif",".png",".tga"]
size = 512

for i,f in enumerate(os.listdir(path)):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    
    img = Image.open(os.path.join(path,f))
    w,h = img.size
    
    if w==h:
        img.resize((size,size)).convert('RGB').save(f'./dataset/1/{i:05}.jpg')
    
    elif w>h:
        img.crop(((w-h)/2,0,(w-h)/2+h,h)).resize((size,size)).convert('RGB').save(f'./dataset/1/{i:05}.jpg')
    
    elif h>w:
        img.crop((0,(h-w)/2,w,(h-w)/2+w)).resize((size,size)).convert('RGB').save(f'./dataset/1/{i:05}.jpg')
        
    if i%100==0:
        print(i)
    

## _Read Images to Array from Folder

In [ ]:
# _Read Images to Array from Folder

from PIL import Image
import os, os.path
import numpy as np

In [ ]:
# _Read Images to Array from Folder

imgs = []
labels = []

path = "/content/fingers/train"
valid_images = [".jpg",".gif",".png",".tga"]

for i,f in enumerate(os.listdir(path)):
  ext = os.path.splitext(f)[1]
  filename = os.path.splitext(f)[0]
  if ext.lower() not in valid_images:
      continue
  imgs.append(np.array(Image.open(os.path.join(path,f))))
  labels.append(int(filename[-2]))
  
  if i%5000==0:
    print(f'processing index {i}')

img_data_x = np.array(imgs)
img_data_y = np.array(labels)

print(f'collected data {img_data_x.shape}\ncollected label {img_data_y.shape}')

## _Plot 25 Images

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# _Plot 25 images by matplotlib.pyplot

plt.figure(figsize=(10,6))
for i in range(15):
    plt.subplot(3,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap='gray')
    plt.xlabel(y_train[i])
plt.show()